# Segmentação e limpeza dos documentos do conjunto de dados CohQuAD Init EN

Realiza o download/cópia do conjunto de dados para segmentação e limpeza. 

Realiza as seguintes limpezas:
- Eliminar pontuações repetidas (???,!!!);
- Eliminar de espaços em branco repetidos;
- Remover documentos com mais de 10 sentenças, pois geram muitas permutações e acabam com a memória;
- Remover documentos que não possuem verbos(regular e auxiliar) ou substantivos;

Após a limpeza segmenta os documentos.

**Gera o arquivo original.csv compactado em original.zip com os documentos originais.**

Cada linha de **original.csv** é formado por `["id","sentencas","documento","classe"]`.
 - `"id"` é o idenficador do documento na base de dados original.
 - `"sentencas"` é uma lista com as sentenças do documento (neste conjunto de dados todo os documentos possuem somente uma sentença). 
 - `"documento"` o documento no formato original.


# 1 Preparação do ambiente

Preparação do ambiente para execução do script.

## 1.1 Tempo inicial de processamento

In [1]:
# Import das bibliotecas.
import time
import datetime

# Marca o tempo de início do processamento
inicio_processamento = time.time()

## 1.2 Funções e classes auxiliares

Verifica se existe o diretório cohebert no diretório corrente.   


In [2]:
# Import das bibliotecas.
import os # Biblioteca para manipular arquivos

# ============================  
def verificaDiretorioCoheBERT():
    """
      Verifica se existe o diretório cohebert no diretório corrente.    
    """
    
    # Verifica se o diretório existe
    if not os.path.exists(DIRETORIO_COHEBERT):  
        # Cria o diretório
        os.makedirs(DIRETORIO_COHEBERT)
        logging.info("Diretório Cohebert criado: {}".format(DIRETORIO_COHEBERT))
    
    return DIRETORIO_COHEBERT

Realiza o download e um arquivo

In [3]:
# Import das bibliotecas.
import requests # Biblioteca de download
from tqdm.notebook import tqdm as tqdm_notebook # Biblioteca para barra de progresso
import os # Biblioteca para manipular arquivos

def downloadArquivo(url_arquivo, nome_arquivo_destino):
    """
      Realiza o download de um arquivo de uma url em salva em nome_arquivo_destino.
    
      Parâmetros:
        `url_arquivo` - URL do arquivo a ser feito download.      
        `nome_arquivo_destino` - Nome do arquivo a ser salvo.      
    """
    
    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
    
    # Realiza o download de um arquivo em uma url
    data = requests.get(url_arquivo, stream=True)
    
    # Verifica se o arquivo existe
    if data.status_code != 200:
        logging.info("Exceção ao tentar realizar download {}. Response {}.".format(url_arquivo, data.status_code))
        data.raise_for_status()
        return

    # Recupera o nome do arquivo a ser realizado o download    
    nome_arquivo = nome_arquivo_destino.split("/")[-1]  

    # Define o nome e caminho do arquivo temporário    
    nome_arquivo_temporario = DIRETORIO_COHEBERT + "/" + nome_arquivo + "_part"
    
    logging.info("Download do arquivo: {}.".format(nome_arquivo_destino))
    
    # Baixa o arquivo
    with open(nome_arquivo_temporario, "wb") as arquivo_binario:        
        tamanho_conteudo = data.headers.get("Content-Length")        
        total = int(tamanho_conteudo) if tamanho_conteudo is not None else None
        # Barra de progresso de download
        progresso_bar = tqdm_notebook(unit="B", total=total, unit_scale=True)                
        # Atualiza a barra de progresso
        for chunk in data.iter_content(chunk_size=1024):        
            if chunk:                
                progresso_bar.update(len(chunk))
                arquivo_binario.write(chunk)
    
    # Renomeia o arquivo temporário para o arquivo definitivo
    os.rename(nome_arquivo_temporario, nome_arquivo_destino)
    
    # Fecha a barra de progresso.
    progresso_bar.close()

Remove tags de um documento

In [4]:
def remove_tags(documento):
    """
      Remove tags de um documento
    """
    
    import re

    documento_limpo = re.compile("<.*?>")
    return re.sub(documento_limpo, "", documento)

Funções auxiliares de arquivos

In [5]:
def carregar(nome_arquivo, encoding="Windows-1252"):
    """
      Carrega um arquivo texto e retorna as linhas como um único parágrafo(texto).
    
      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser carregado.  
    """

    # Abre o arquivo
    arquivo = open(nome_arquivo, "r", encoding= encoding)
    
    paragrafo = ""
    for linha in arquivo:
        linha = linha.splitlines()
        linha = " ".join(linha)
        # Remove as tags existentes no final das linhas
        linha = remove_tags(linha)
        if linha != "":
          paragrafo = paragrafo + linha.strip() + " "
    
    # Fecha o arquivo
    arquivo.close()

    # Remove os espaços em branco antes e depois do parágrafo
    return paragrafo.strip()

In [6]:
def carregarLista(nome_arquivo, encoding="Windows-1252"):
    """
      Carrega um arquivo texto e retorna as linhas como uma lista de sentenças(texto).
    
      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser carregado.   
        `encoding` - Codificação dos caracteres do arquivo.
    """

    # Abre o arquivo
    arquivo = open(nome_arquivo, "r", encoding= encoding)
    
    sentencas = []
    for linha in arquivo:        
        linha = linha.splitlines()
        linha = " ".join(linha)
        linha = remove_tags(linha)
        if linha != "":
          sentencas.append(linha.strip())
    
    # Fecha o arquivo
    arquivo.close()

    return sentencas 

In [7]:
def salvar(nome_arquivo,texto):                       
    """
      Salva um texto em arquivo.
     
      Parâmetros:
        `nome_arquivo` - Nome do arquivo a ser salvo.
        `texto` - Texto a ser salvo.     
    """

    arquivo = open(nome_arquivo, "w")
    arquivo.write(str(texto))
    arquivo.close()

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [8]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempo_arredondado = int(round((tempo)))
    
    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempo_arredondado))    

Classe(ModelArguments) de definição dos parâmetros do modelo

In [9]:
# Import das bibliotecas.
from dataclasses import dataclass, field
from typing import Dict, Optional
from typing import List

@dataclass
class ModeloArgumentosMedida:
    max_seq_len: Optional[int] = field(
        default=None,
        metadata={"help": "max seq len"},
    )    
    pretrained_model_name_or_path: str = field(
        default="neuralmind/bert-base-portuguese-cased",
        metadata={"help": "nome do modelo pré-treinado do BERT."},
    )
    modelo_spacy: str = field(
        default="pt_core_news_lg",
        metadata={"help": "nome do modelo do spaCy."},
    )
    versao_modelo_spacy: str = field(
        default="-3.2.0",
        metadata={"help": "versão do nome do modelo no spaCy."},
    )
    sentenciar_documento: bool = field(
        default=True,
        metadata={"help": "Dividir o documento em sentenças(frases)."},
    )
    do_lower_case: bool = field(
        default=False,
        metadata={"help": "define se o texto do modelo deve ser todo em minúsculo."},
    )    
    output_attentions: bool = field(
        default=False,
        metadata={"help": "habilita se o modelo retorna os pesos de atenção."},
    )
    output_hidden_states: bool = field(
        default=False,
        metadata={"help": "habilita gerar as camadas ocultas do modelo."},
    )
    usar_mcl_ajustado : bool = field(
        default=False,
        metadata={"help": "habilita o carragamento de mcl ajustado."},
    )

Biblioteca de limpeza de tela


In [10]:
# Import das bibliotecas.
from IPython.display import clear_output

## 1.3 Tratamento de logs

In [11]:
# Import das bibliotecas.
import logging # Biblioteca de logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s")

logger = logging.getLogger()
logger.setLevel(logging.INFO)

## 1.4 Identificando o ambiente Colab

In [12]:
# Import das bibliotecas.
import sys # Biblioteca para acessar módulos do sistema

# Se estiver executando no Google Colaboratory
# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = "google.colab" in sys.modules

## 1.5 Colaboratory

Usando Colab GPU para Treinamento


Uma GPU pode ser adicionada acessando o menu e selecionando:

`Edit -> Notebook Settings -> Hardware accelerator -> (GPU)`

Em seguida, execute a célula a seguir para confirmar que a GPU foi detectada.

In [13]:
# Import das bibliotecas.
import tensorflow as tf

# Recupera o nome do dispositido da GPU.
device_name = tf.test.gpu_device_name()

# O nome do dispositivo deve ser parecido com o seguinte:
if device_name == "/device:GPU:0":
    logging.info("Encontrei GPU em: {}".format(device_name))
else:
    logging.info("Dispositivo GPU não encontrado")
    #raise SystemError("Dispositivo GPU não encontrado")

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:root:Dispositivo GPU não encontrado


Nome da GPU

Para que a torch use a GPU, precisamos identificar e especificar a GPU como o dispositivo. Posteriormente, em nosso ciclo de treinamento, carregaremos dados no dispositivo.

Vale a pena observar qual GPU você recebeu. A GPU Tesla P100 é muito mais rápido que as outras GPUs, abaixo uma lista ordenada:
- 1o Tesla P100
- 2o Tesla T4
- 3o Tesla P4 (Não tem memória para execução 4 x 8, somente 2 x 4)
- 4o Tesla K80 (Não tem memória para execução 4 x 8, somente 2 x 4)

In [14]:
# Import das bibliotecas.
import torch

def getDeviceGPU():
    """
      Retorna um dispositivo de GPU se disponível ou CPU.
    
      Retorno:
        `device` - Um device de GPU ou CPU.       
    """
        
    # Se existe GPU disponível.
    if torch.cuda.is_available():
        
        # Diz ao PyTorch para usar GPU.    
        device = torch.device("cuda")
        
        logging.info("Existem {} GPU(s) disponíveis.".format(torch.cuda.device_count()))
        logging.info("Iremos usar a GPU: {}.".format(torch.cuda.get_device_name(0)))

    # Se não.
    else:        
        logging.info("Sem GPU disponível, usando CPU.")
        device = torch.device("cpu")
        
    return device

In [15]:
# Recupera o device com GPU ou CPU
device = getDeviceGPU()

INFO:root:Sem GPU disponível, usando CPU.


Memória

Memória disponível no ambiente

In [16]:
# Importando as bibliotecas.
from psutil import virtual_memory

ram_gb = virtual_memory().total / 1e9
logging.info("Seu ambiente de execução tem {: .1f} gigabytes de RAM disponível\n".format(ram_gb))

if ram_gb < 20:
  logging.info("Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> \"Alterar tipo de tempo de execução\"")
  logging.info("e selecione High-RAM. Então, execute novamente está célula")
else:
  logging.info("Você está usando um ambiente de execução de memória RAM alta!")

INFO:root:Seu ambiente de execução tem  13.6 gigabytes de RAM disponível

INFO:root:Para habilitar um tempo de execução de RAM alta, selecione menu o ambiente de execução> "Alterar tipo de tempo de execução"
INFO:root:e selecione High-RAM. Então, execute novamente está célula


## 1.6 Monta uma pasta no google drive para carregar os arquivos de dados.

In [17]:
# import necessário
from google.colab import drive

# Monta o drive na pasta especificada
drive.mount("/content/drive")

Mounted at /content/drive


## 1.7 Instalação do wandb

Instalação

In [18]:
!pip install --upgrade wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 7.4 MB/s 
     |████████████████████████████████| 181 kB 70.5 MB/s 
     |████████████████████████████████| 162 kB 70.3 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 158 kB 59.7 MB/s 
     |████████████████████████████████| 157 kB 51.9 MB/s 
     |████████████████████████████████| 157 kB 62.2 MB/s 
     |████████████████████████████████| 157 kB 51.1 MB/s 
     |████████████████████████████████| 157 kB 58.8 MB/s 
     |████████████████████████████████| 157 kB 58.1 MB/s 
     |████████████████████████████████| 157 kB 59.1 MB/s 
     |████████████████████████████████| 157 kB 60.0 MB/s 
     |████████████████████████████████| 156 kB 75.7 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=56b7e811dbe2c469f83c59688373676a7261affc5f049354c040182d5f96

## 1.8 Instalação do spaCy

https://spacy.io/

Modelos do spaCy para português:
https://spacy.io/models/pt

In [19]:
# Instala o spacy
!pip install -U pip setuptools wheel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 7.3 MB/s 
     |████████████████████████████████| 1.2 MB 58.4 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
numba 0.56.2 requires setuptools<60, but you have setuptools 65.4.0 which is incompatible.


In [20]:
# Instala uma versão específica
!pip install -U spacy==3.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 89.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.6/660.6 kB 51.4 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.1.1
    Uninstalling typing_extensions-4.1.1:
      Successfully uninstalled typing_extensions-4.1.1
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.9.2
    Uninstalling pydantic-1.9.2:
      Successfully uninstalled pydantic-1.9.2
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.0
    Uninstalling thinc-8.1.0:
      Successfully uninstalled thinc-8.1.0
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.1
    Uninstalling spacy-3.4.1:
      Successfully uninstalled spacy-3.4.1
ERROR: p

# 2 Parametrização

## Gerais

In [21]:
# Definição dos parâmetros a serem avaliados

## Específicos

Parâmetros do modelo

In [22]:
# Definição dos parâmetros do Modelo.
model_args = ModeloArgumentosMedida(     
    max_seq_len = 512,
    #pretrained_model_name_or_path = "https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-large-portuguese-cased/bert-large-portuguese-cased_pytorch_checkpoint.zip",
    #pretrained_model_name_or_path = "https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-base-portuguese-cased/bert-base-portuguese-cased_pytorch_checkpoint.zip",
    
    pretrained_model_name_or_path = "bert-large-cased",
    #pretrained_model_name_or_path = "bert-base-cased"
    #pretrained_model_name_or_path = "neuralmind/bert-large-portuguese-cased",
    #pretrained_model_name_or_path = "neuralmind/bert-base-portuguese-cased",    
    #pretrained_model_name_or_path = "bert-base-multilingual-cased",
    #pretrained_model_name_or_path = "bert-base-multilingual-uncased",

    modelo_spacy = "en_core_web_lg",
    #modelo_spacy = "en_core_web_md",
    #modelo_spacy = "en_core_web_sm",
    #modelo_spacy = "pt_core_news_lg",
    #modelo_spacy = "pt_core_news_md",
    #modelo_spacy = "pt_core_news_sm",

    versao_modelo_spacy = "3.2.0",
    sentenciar_documento = False,
    do_lower_case = False, # default True  
    output_attentions = False, # default False
    output_hidden_states = True, # default False, se True retorna todas as camadas do modelo para as operações de soma e concatenação
    usar_mcl_ajustado = False, # Especifica se deve ser carregado um MCL ajustado ou pré-treinado. Necessário especificar o tipo do modelo em pretrained_model_name_or_path. 
)

## Nome do diretório do cohebert

In [23]:
# Diretório do cohebert
DIRETORIO_COHEBERT = "COHQUAD_INIT_EN"

## Define o caminho para os arquivos de dados

In [24]:
# Diretório local para os arquivos pré-processados
DIRETORIO_LOCAL = "/content/" + DIRETORIO_COHEBERT + "/"

# Diretório no google drive com os arquivos pré-processados
DIRETORIO_DRIVE = "/content/drive/MyDrive/Colab Notebooks/Data/" + DIRETORIO_COHEBERT + "/"

# 3 spaCy

## 3.1 Download arquivo modelo

https://spacy.io/models/pt

### Função download modelo spaCy

In [25]:
def downloadSpacy(model_args):
    """
      Realiza o download do arquivo do modelo para o diretório corrente.
    
      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.       
    """
    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
        
    # Nome arquivo spacy
    ARQUIVO_MODELO_SPACY = model_args.modelo_spacy
    # Versão spaCy
    VERSAO_SPACY = "-" + model_args.versao_modelo_spacy
    # Nome arquivo compactado
    NOME_ARQUIVO_MODELO_COMPACTADO = ARQUIVO_MODELO_SPACY + VERSAO_SPACY + ".tar.gz"
    
    # Url do arquivo
    URL_ARQUIVO_MODELO_COMPACTADO = "https://github.com/explosion/spacy-models/releases/download/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + "/" + NOME_ARQUIVO_MODELO_COMPACTADO

    # Realiza o download do arquivo do modelo
    logging.info("Download do arquivo do modelo do spaCy.")
    downloadArquivo(URL_ARQUIVO_MODELO_COMPACTADO, DIRETORIO_COHEBERT + "/" + NOME_ARQUIVO_MODELO_COMPACTADO)

## 3.2 Descompacta o arquivo do modelo

### Função descompacta modelo spaCy

In [26]:
# Import das bibliotecas.
import tarfile # Biblioteca de descompactação

def descompactaSpacy(model_args):
    """
      Descompacta o arquivo do modelo.
    
      Parâmetros:
        `model_args` - Objeto com os argumentos do modelo.       
    """
    
    # Verifica se existe o diretório base do cohebert e retorna o nome do diretório
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
    
    # Nome arquivo spacy
    ARQUIVO_MODELO_SPACY = model_args.modelo_spacy
    # Versão spaCy
    VERSAO_SPACY = "-" + model_args.versao_modelo_spacy
    
    # Nome do arquivo a ser descompactado
    NOME_ARQUIVO_MODELO_COMPACTADO = DIRETORIO_COHEBERT + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + ".tar.gz"
    
    logging.info("Descompactando o arquivo do modelo do spaCy.")
    arquivo_tar = tarfile.open(NOME_ARQUIVO_MODELO_COMPACTADO, "r:gz")    
    arquivo_tar.extractall(DIRETORIO_COHEBERT)    
    arquivo_tar.close()
    
    # Apaga o arquivo compactado
    if os.path.isfile(NOME_ARQUIVO_MODELO_COMPACTADO):        
        os.remove(NOME_ARQUIVO_MODELO_COMPACTADO)

## 3.3 Carrega o modelo

### Função carrega modelo spaCy

In [27]:
# Import das bibliotecas.
import spacy # Biblioteca do spaCy

def carregaSpacy(model_args):
    """
    Realiza o carregamento do Spacy.
    
    Parâmetros:
      `model_args` - Objeto com os argumentos do modelo.           
    """
    
    # Verifica se existe o diretório base
    DIRETORIO_COHEBERT = verificaDiretorioCoheBERT()
                  
    # Nome arquivo spacy
    ARQUIVO_MODELO_SPACY = model_args.modelo_spacy
    # Versão spaCy
    VERSAO_SPACY = "-" + model_args.versao_modelo_spacy
    # Caminho raoz do modelo do spaCy
    DIRETORIO_MODELO_SPACY =  DIRETORIO_COHEBERT + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY

    # Verifica se o diretório existe
    if os.path.exists(DIRETORIO_MODELO_SPACY) == False:
        # Realiza o download do arquivo modelo do spaCy
        downloadSpacy(model_args)
        # Descompacta o spaCy
        descompactaSpacy(model_args)

    # Diretório completo do spaCy
    DIRETORIO_MODELO_SPACY = DIRETORIO_COHEBERT + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + "/" + ARQUIVO_MODELO_SPACY + "/" + ARQUIVO_MODELO_SPACY + VERSAO_SPACY + "/"

    # Carrega o spaCy. Necessário somente "tagger" para encontrar os substantivos
    nlp = spacy.load(DIRETORIO_MODELO_SPACY)
    logging.info("spaCy carregado.")

    # Retorna o spacy carregado
    return nlp 

### Carrega o modelo spaCy


In [28]:
# Carrega o modelo spaCy
nlp = carregaSpacy(model_args)

INFO:root:Diretório Cohebert criado: COHQUAD_INIT_EN
INFO:root:Download do arquivo do modelo do spaCy.
INFO:root:Download do arquivo: COHQUAD_INIT_EN/en_core_web_lg-3.2.0.tar.gz.


  0%|          | 0.00/777M [00:00<?, ?B/s]

INFO:root:Descompactando o arquivo do modelo do spaCy.
INFO:root:spaCy carregado.


## 3.4 Funções auxiliares spaCy

### getStopwords

Recupera as stopwords do spaCy

In [29]:
def getStopwords(nlp):
    """
      Recupera as stop words do nlp(Spacy).
    
      Parâmetros:
        `nlp` - Um modelo spaCy carregado.           
    """
    
    spacy_stopwords = nlp.Defaults.stop_words

    return spacy_stopwords 

Lista dos stopwords

In [30]:
logging.info("Quantidade de stopwords: {}.".format(len(getStopwords(nlp))))

print(getStopwords(nlp))

INFO:root:Quantidade de stopwords: 326.


{'all', 'onto', 'we', 'someone', 'full', 'four', '’ve', 'former', 'less', 'whom', 'sixty', 'few', 'always', 'anyway', 'together', 'between', 'put', '‘re', 'again', 'does', 'several', 'beforehand', 'sometimes', 'whenever', '‘d', 'everywhere', 'himself', 'must', 'can', "'ve", 'upon', 'never', 'nor', 'you', '’m', 'front', 'which', 'its', 'thereafter', 'across', 'when', 'please', 'toward', 'no', 'his', 'after', 'hereby', 'she', 'anything', 'third', 'behind', 'ours', 'quite', 'though', 'under', '‘m', 'make', 'wherein', 'of', 'afterwards', 'too', 'therefore', 'mine', 'empty', 'part', 'her', 'nevertheless', 'others', 'down', 'moreover', 'latter', 'really', 'somewhere', 'fifty', 'move', 'against', 'n‘t', 'thereupon', 'without', 'bottom', 'not', 'further', 'as', 'ourselves', 'below', 'wherever', 'used', 'should', 'becomes', 'done', 'an', 'something', 'due', 'per', 'perhaps', 'back', 'up', 'over', 'he', 'twelve', 'did', 'why', 'within', 'through', 'another', 'anywhere', 'twenty', 'already', 'bes

### getVerbos
Localiza os verbos da sentença

In [31]:
# Import das bibliotecas.
import spacy   
from spacy.util import filter_spans
from spacy.matcher import Matcher

# (verbo normal como auxilar ou auxilar) + vários verbos auxiliares +verbo principal ou verbo auxiliar
gramaticav1 =  [
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["aux","aux:pass"]}},  #verbo auxiliar                                  
                {"POS": "VERB", "OP": "?", "DEP": {"IN": ["ROOT","aux","xcomp","aux:pass"]}},  #verbo normal como auxiliar
                {"POS": "AUX", "OP": "*", "DEP": {"IN": ["aux","xcomp","aux:pass"]}},  #verbo auxiliar   
                {"POS": "VERB", "OP": "+"}, #verbo principal
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["cop","aux","xcomp","aux:pass"]}},  #verbo auxiliar
               ] 

# verbo auxiliar + verbo normal como auxiliar + conjunção com preposição + verbo
gramaticav2 =  [               
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["aux","aux:pass"]}},  #verbo auxiliar                   
                {"POS": "VERB", "OP": "+", "DEP": {"IN": ["ROOT"]}},  #verbo principal       
                {"POS": "SCONJ", "OP": "+", "DEP": {"IN": ["mark"]}}, #conjunção com preposição
                {"POS": "VERB", "OP": "+", "DEP": {"IN": ["xcomp"]}}, #verbo normal como complementar
               ] 

#Somente verbos auxiliares
gramaticav3 =  [
                {"POS": "AUX", "OP": "?"},  #Verbos auxiliar 
                {"POS": "AUX", "OP": "?", "DEP": {"IN": ["cop"]}},  #Verbos auxiliar de ligação (AUX+(cop))
                {"POS": "ADJ", "OP": "+", "DEP": {"IN": ["ROOT"]}}, 
                {"POS": "AUX", "OP": "?"}  #Verbos auxiliar 
               ] 

matcherv = Matcher(nlp.vocab)
         
matcherv.add("frase verbal", [gramaticav1])
matcherv.add("frase verbal", [gramaticav2])
matcherv.add("frase verbal", [gramaticav3])

#Retorna a Frase Verbal
def getVerbos(periodo):    
  #Processa o período
  doc1 = nlp(periodo.text)
  
  # Chama o mather para encontrar o padrão
  matches = matcherv(doc1)

  padrao = [doc1[start:end] for _, start, end in matches]

  #elimina as repetições e sobreposições
  #return filter_spans(padrao)
  lista1 = filter_spans(padrao)

  # Converte os itens em string
  lista2 = []
  for x in lista1:
      lista2.append(str(x))
  
  return lista2

### getDicPOSQtde

Conta as POS Tagging de uma sentença

In [32]:
def getDicPOSQtde(sentenca):

    # Verifica se o sentenca não foi processado pelo spaCy  
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Retorna inteiros que mapeiam para classes gramaticais
  conta_dicionarios = doc.count_by(spacy.attrs.IDS["POS"])

  # Dicionário com as tags e quantidades
  novodic = dict()
  
  for pos, qtde in conta_dicionarios.items():
    classe_gramatical = doc.vocab[pos].text
    novodic[classe_gramatical] = qtde

  return novodic

In [33]:
def getDicTodasPOSQtde(sentenca):

    # Verifica se o sentenca não foi processado pelo spaCy  
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Retorna inteiros que mapeiam para classes gramaticais
  conta_dicionarios = doc.count_by(spacy.attrs.IDS["POS"])

  # Dicionário com as tags e quantidades    
  novo_dic = {"PRON":0, "VERB":0, "PUNCT":0, "DET":0, "NOUN":0, "AUX":0, "CCONJ":0, "ADP":0, "PROPN":0, "ADJ":0, "ADV":0, "NUM":0, "SCONJ":0, "SYM":0, "SPACE":0, "INTJ":0, "X": 0}
    
  for pos, qtde in conta_dicionarios.items():
    classe_gramatical = doc.vocab[pos].text
    novodic[classe_gramatical] = qtde

  return novo_dic

### getSomaDic

Soma os valores de dicionários com as mesmas chaves.

In [34]:
# Import das bibliotecas.
from collections import Counter
from functools import reduce

def atualizaValor(a,b):
    a.update(b)
    return a

def getSomaDic(lista):
    
  # Soma os dicionários da lista
  novodic = reduce(atualizaValor, (Counter(dict(x)) for x in lista))
 
  return novodic

### getTokensSentenca

Retorna a lista de tokens da sentenca.

In [35]:
def getTokensSentenca(sentenca):

    # Verifica se o sentenca não foi processado pelo spaCy  
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Lista dos tokens
  lista = []

  # Percorre a sentença adicionando os tokens
  for token in doc:    
    lista.append(token.text)

  return lista

### getPOSTokensSentenca

Retorna a lista das POS-Tagging dos tokens da sentenca.

In [36]:
def getPOSTokensSentenca(sentenca):

  # Verifica se o sentenca não foi processado pelo spaCy  
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Lista dos tokens
  lista = []

  # Percorre a sentença adicionando os tokens
  for token in doc:    
    lista.append(token.pos_)

  return lista

### getListaTokensPOSSentenca

Retorna duas listas uma com os tokens e a outra com a POS-Tagging dos tokens da sentenca.

In [37]:
def getListaTokensPOSSentenca(sentenca):
  # Verifica se o sentenca não foi processado pelo spaCy  
  if type(sentenca) is not spacy.tokens.doc.Doc:
      # Realiza o parsing no spacy
      doc = nlp(sentenca)
  else:
      doc = sentenca

  # Lista dos tokens
  lista_tokens = []
  lista_pos = []

  # Percorre a sentença adicionando os tokens e as POS
  for token in doc:    
    lista_tokens.append(token.text)
    lista_pos.append(token.pos_)
    
  return lista_tokens, lista_pos

### Tadução das tags

Tags de palavras universal

https://universaldependencies.org/u/pos/

Detalhes das tags em português:
http://www.dbd.puc-rio.br/pergamum/tesesabertas/1412298_2016_completo.pdf

In [38]:
#dicionário que contêm pos tag universal e suas explicações
palavra_universal_dict = {
  "X"    : "Outro",
  "VERB" : "Verbo ",
  "SYM"  : "Símbolo",
  "CONJ" : "Conjunção",
  "SCONJ": "Conjunção subordinativa",
  "PUNCT": "Pontuação",
  "PROPN": "Nome próprio",
  "PRON" : "Pronome substativo",
  "PART" : "Partícula, morfemas livres",
  "NUM"  : "Numeral",
  "NOUN" : "Substantivo",
  "INTJ" : "Interjeição",
  "DET"  : "Determinante, Artigo e pronomes adjetivos",
  "CCONJ": "Conjunção coordenativa",
  "AUX"  : "Verbo auxiliar",
  "ADV"  : "Advérbio",
  "ADP"  : "Preposição",
  "ADJ"  : "Adjetivo"
}
  
#Explica a POS
def getPOSPalavraUniversalTraduzido(palavra):
  if palavra in palavra_universal_dict.keys():
      traduzido = palavra_universal_dict[palavra]
  else:
      traduzido = "NA" 
  return traduzido

# 4 Pré-processamento do arquivo de dados


## 4.1 Cria o diretório para receber os arquivos de dados

In [39]:
# Importando as bibliotecas.
import os

# Cria o diretório para receber os arquivos Originais e Permutados
# Diretório a ser criado
dirbase = DIRETORIO_LOCAL[:-1]

if not os.path.exists(dirbase):  
    # Cria o diretório
    os.makedirs(dirbase)    
    logging.info("Diretório criado: {}".format(dirbase))
else:    
    logging.info("Diretório já existe: {}".format(dirbase))

INFO:root:Diretório já existe: /content/COHQUAD_INIT_EN


## 4.2 Especifica os nomes dos arquivos de dados



In [40]:
# Nome do arquivo
NOME_ARQUIVO_FONTE = "cohebert.csv"

## 4.3 Copia o arquivos de dados bruto do google drive

In [41]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:

  !cp "$DIRETORIO_DRIVE$NOME_ARQUIVO_FONTE" "$DIRETORIO_LOCAL"
  
  logging.info("Terminei a cópia!")

INFO:root:Terminei a cópia!


## 4.4 Carrega os dados no formato bruto

Atributos do arquivo **COHEBERT**:
0. "id"
1. "documento"
2. "classe" - coerente(0) ou incoerente(1). 

In [42]:
# Import das bibliotecas
import pandas as pd

# Abre o arquivo e retorna o DataFrame
lista_documentos_fonte = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_FONTE, sep=";", encoding="UTF-8")

logging.info("TERMINADO FONTES: {}.".format(len(lista_documentos_fonte)))

INFO:root:TERMINADO FONTES: 40.


In [43]:
lista_documentos_fonte.sample(5)

,id,documento,classe
22,12,What is a stack and how to pop an element from...,1
9,5_pert_0,How to push elements in a queue data structure?,0
35,18_pert_0,How are the operations to enqueue and dequeue ...,0
16,9,What is a stack and how to push its element?,1
20,11,What is a queue and how to dequeue an element ...,1


Renomeia o dataset e selecionamento somente os documentos originais coerentes.


In [44]:
df = lista_documentos_fonte[lista_documentos_fonte['classe']==1]

In [45]:
df.sample(5)

,id,documento,classe
2,2,How to dequeue elements in a queue?,1
32,17,How are the operations to push and pop element...,1
22,12,What is a stack and how to pop an element from...,1
38,20,In a queue does the enqueue operation occur at...,1
14,8,How to pop elements in a stack data structure?,1


## 4.5 Processando os documentos originais

### 4.5.1 Pré-processamento

- Eliminar pontuações repetidas (???,!!!);
- Eliminar de espaços em branco repetidos;
- Remover documentos com mais de 10 sentenças, pois geram muitas permutações e acabam com a memória;
- Remover documentos que não possuem verbos(regular e auxiliar) ou substantivos.


In [46]:
# Import das bibliotecas,
import re # Biblioteca para expressões regulares
from tqdm.notebook import tqdm as tqdm_notebook # Biblioteca para barra de progresso

# Lista para os documentos tratados
documentos_tratados = []

# Acumula o total de sentenças
conta_barra_n = 0
total_sentencas = 0
conta_interrogacoes = 0
conta_sem_interrogacoes = 0
conta_espacos = 0
conta_documento_2sentencas = 0
conta_documentos_10sentencas = 0
conta_sem_classes_minimo = 0

# Barra de progresso dos dados
dados_bar = tqdm_notebook(df.iterrows(), desc=f"Dados", unit=f"registro", total=len(df))

# Percorre os registros dos dados bruto
for (i, linha) in dados_bar:
  
  # Recupera o id documento
  id_documento = linha.values[0]

  # Recupera o documento
  documento = str(linha.values[1])
  
  # Substitui \n por espaço em branco no documento
  conta_caracter_barra_n = documento.count("\n")
  if conta_caracter_barra_n > 0:
    # Transforma \n em espaços em branco 
    documento = documento.replace("\n"," ")
    conta_barra_n = conta_barra_n + 1

  # Transforma em string e remove os espaços do início e do fim
  documento = str(documento).strip()

  # Conta sentenças com duas ou mais interrogação
  conta_caracter_interrogacoes = documento.count("?")
  if conta_caracter_interrogacoes > 1:
    # Transforma 2 ou mais interrogações consecutivas em 1
    documento = re.sub("\?+", "?", documento)
    conta_interrogacoes = conta_interrogacoes + 1
  
  # Conta sentenças sem interrogação
  if conta_caracter_interrogacoes == 0:
      conta_sem_interrogacoes = conta_sem_interrogacoes + 1

  # Conta caracteres em branco repetidos
  conta_caracter_espacos = documento.count("  ")
  if conta_caracter_espacos > 0:
    # Transforma 2 ou mais interrogações consecutivas em 1    
    documento = re.sub("\W*\?+\W*", "? ", documento)
    conta_espacos = conta_espacos + 1
 
  # Aplica sentenciação do spacy no documento
  doc = nlp(documento) 

  # Conta o número de sentenças do documento
  conta_sentencas = len(list(doc.sents))
  if conta_sentencas > 1:
    conta_documento_2sentencas = conta_documento_2sentencas + 1

  # Conta os documentos com mais de 10 sentenças
  if conta_sentencas > 10:
    conta_documentos_10sentencas = conta_documentos_10sentencas + 1
    print("Texto com mais de 10 sentenças:",documento)

  # Conta o total de sentenças do documento
  total_sentencas = total_sentencas + conta_sentencas
  
  # Verifica se a sentença possui no mínimo um verbo(regular ou auxiliar) ou substantivo
  classe_pos_minimo = ["VERB", "NOUN", "AUX"]
  pos_minimo = False
  # Recupera a lista de tokens e pos do documento
  lista_tokens, lista_postagging = getListaTokensPOSSentenca(doc)
  for i, pos in enumerate(lista_postagging):
      if pos in classe_pos_minimo:
        pos_minimo = True

  if pos_minimo == False:
    conta_sem_classes_minimo = conta_sem_classes_minimo + 1

  # Adiciona textos somente com menos de 10 sentenças e possui no mínimo um verbo ou um substantivo
  if conta_sentencas < 10 and pos_minimo == True:
    # Adiciona a linha tratada com o id do documento, o documento a lista
    documentos_tratados.append([id_documento, documento])

Dados:   0%|          | 0/20 [00:00<?, ?registro/s]

In [47]:
print("Total de documentos com \\n                        :", conta_barra_n)
print("Total de documentos com 2 ou mais interrogações   :", conta_interrogacoes)
print("Total de documentos sem interrogações             :", conta_sem_interrogacoes)
print("Total de documentos com 2 ou mais espaços         :", conta_espacos)
print("Total de documentos com 2 ou mais sentenças       :", conta_documento_2sentencas)
print("Total de documentos com mais de 10 sentenças      :", conta_documentos_10sentencas)
print("Total de documentos sem verbo ou substantivo      :", conta_sem_classes_minimo)
print("Total de sentenças nos documentos tratados        :", total_sentencas)
print("Total de documentos tratadas                      :", len(documentos_tratados))

Total de documentos com \n                        : 0
Total de documentos com 2 ou mais interrogações   : 0
Total de documentos sem interrogações             : 0
Total de documentos com 2 ou mais espaços         : 0
Total de documentos com 2 ou mais sentenças       : 0
Total de documentos com mais de 10 sentenças      : 0
Total de documentos sem verbo ou substantivo      : 0
Total de sentenças nos documentos tratados        : 20
Total de documentos tratadas                      : 20


In [48]:
# Import das bibliotecas.
import pandas as pd

# Cria o dataframe da lista
df_documentos_tratados = pd.DataFrame(documentos_tratados, columns = ["id", "documento"])

#Mostra o número de documentos carregados
print("%d linhas carregadas do arquivo" % (len(df_documentos_tratados)))

# Mostra 10 linhas aleatórias dos dados
df_documentos_tratados.sample(10)

20 linhas carregadas do arquivo


,id,documento
19,20,In a queue does the enqueue operation occur at...
5,6,How to push and pop elements in a stack data s...
15,16,What is a queue and how to enqueue and dequeue...
1,2,How to dequeue elements in a queue?
3,4,How to push and pop elements in a stack?
11,12,What is a stack and how to pop an element from...
8,9,What is a stack and how to push its element?
14,15,What is a stack and how to push and pop its el...
12,13,What is a queue and how to enqueue an element ...
13,14,What is a stack and how to push an element on it?


Todos documentos são formadas por uma única sentença. O Spacy realiza a divisão incorretamente dos documentos devido a caracteres e letras em maiúsculo.

In [49]:
# Import das bibliotecas
import re
# Biblioteca para barra de progresso
from tqdm.notebook import tqdm as tqdm_notebook

# Barra de progresso dos dados
dados_bar = tqdm_notebook(df_documentos_tratados.iterrows(), desc=f"Dados", unit=f"registro", total=len(df_documentos_tratados))

# Percorre os registros dos dados bruto
for (i, linha) in dados_bar:

  # Recupera o documento
  documento = str(linha.values[1])  

  # Aplica sentenciação do spacy no documento
  doc = nlp(documento) 

  sentencas = []    
  # Percorre as sentenças do documento
  for sentenca in doc.sents:                  
      # Adiciona a sentença tratada na lista      
      sentencas.append(str(sentenca))

  conta_sentencas = len(list(doc.sents))
  if conta_sentencas > 1:
    conta_documento_2sentencas = conta_documento_2sentencas + 1
    print(conta_sentencas, " sentenças:", documento)
    print("          Lista das sentenças:", sentencas)

Dados:   0%|          | 0/20 [00:00<?, ?registro/s]

### 4.5.2 Sentenciação do documento

In [50]:
# Import das bibliotecas.
import os # Biblioteca para acessar o sistema de arquivos
from tqdm.notebook import tqdm as tqdm_notebook # Biblioteca para barra de progresso

print("Processando",len(df_documentos_tratados),"documentos tratados")

# Sentenças por documento
lista_documentos_originais = []
total_sentencas_documento = 0
total_documentos_sentenca_uma_palavra = 0
total_documentos_ao_menos_uma_sentenca_uma_palavra = 0

# Documentos excluídos
documentos_excluidos = []

# Barra de progresso dos dados
dados_bar = tqdm_notebook(df_documentos_tratados.iterrows(), desc=f"Dados", unit=f"registro", total=len(df_documentos_tratados))

# Percorre os registros dos documento
for i, linha_documento in dados_bar:
 
  # Recupera o documento
  documento = str(linha.values[1])  

  # Aplica sentenciação do spacy no documento
  doc = nlp(documento) 
  
  # Sequência das sentenças no documento
  sequencia = 1
  total_palavras = 0  
  sentencas = []
  total_sentencas = 0
  sentencas_com_uma_palavra = 0
  sentencas_com_mais_de_uma_palavra = 0
  
  # Percorre as sentenças do documento
  for sentenca in doc.sents:     
       
      # Conta as sentenças do documento
      total_sentencas = total_sentencas + 1
      total_sentencas_documento = total_sentencas_documento + 1
      
      # Transforma em string e remove os espaços do início e do fim
      sentenca1 = str(sentenca).strip()
      
      # Adiciona a sentença tratada na lista      
      sentencas.append(str(sentenca1))
                  
      # Incrementa a sequência da sentença no documento
      sequencia = sequencia + 1
      
      # Percorre a sentença procurando os tokens da sentença
      sentence_tokens = [token.text for token in sentenca]
      
      # Quantidade de palavras por sentença
      qtdePalavra = len(sentence_tokens)
      
      # Verifica se a sentença tem 1 palavra para excluir o documento
      if qtdePalavra == 1:
        sentencas_com_uma_palavra = sentencas_com_uma_palavra + 1
      else:
        sentencas_com_mais_de_uma_palavra = sentencas_com_mais_de_uma_palavra + 1

      # Acumula a quantidade de palavras da sentença
      total_palavras = total_palavras + qtdePalavra

  # Verifica se o documento tem mais de 1 palavra
  if total_palavras > 1:      

    # Se todas as sentencas possui mais de uma palavra  
    if sentencas_com_mais_de_uma_palavra == len(sentencas):      
      if model_args.sentenciar_documento == True:
        # Adiciona o documento na lista com a sentenciação
        lista_documentos_originais.append([linha_documento[0], sentencas, str(linha_documento[1])])
      else:        
        # Adiciona o documento na lista sem a sentenciação
        lista_documentos_originais.append([linha_documento[0], [str(linha_documento[1])], str(linha_documento[1])])
    else:
      # Se existe pelo menos uma sentença com uma palavra e outras sentencas com mais palavras e não é necessário sentenciar
      if sentencas_com_uma_palavra > 0 and sentencas_com_uma_palavra < len(sentencas):
          if model_args.sentenciar_documento == False:    
            # Adiciona o documento na lista sem a sentenciação
            lista_documentos_originais.append([linha_documento[0], [str(linha_documento[1])], str(linha_documento[1])])
          else:        
            print("Documento não adicionado pois possui uma sentença com uma palavra e é necessário sentenciar.")        
            print(" linha_documento:",linha_documento)
            documentos_excluidos.append(linha_documento)
            total_documentos_ao_menos_uma_sentenca_uma_palavra = total_documentos_ao_menos_uma_sentenca_uma_palavra + 1
      else:
          # Se todas as sentenças possuem 1 palavra
          if sentencas_com_uma_palavra == len(sentencas):    
              print("Documento não adicionado pois todas as sentenças possui uma palavra.")        
              print(" linha_documento:",linha_documento)
              documentos_excluidos.append(linha_documento)
              total_documentos_sentenca_uma_palavra = total_documentos_sentenca_uma_palavra + 1
  else:
      print("Documento não adicionado pois possui somente uma palavra.")
      print(" linha_documento:",linha_documento)
      documentos_excluidos.append(linha_documento)

Processando 20 documentos tratados


Dados:   0%|          | 0/20 [00:00<?, ?registro/s]

In [51]:
print("Total de documentos processados                                                :", len(lista_documentos_originais))
print("Total de documentos com sentenças com uma palavra                              :", total_documentos_sentenca_uma_palavra)
print("Total de documentos com pelo menos uma sentença com uma palavra(sem sentenciar):", total_documentos_ao_menos_uma_sentenca_uma_palavra)
print("Total de documentos excluídos                                                  :", len(documentos_excluidos))
print("Total sentenças nos documentos usando spaCy                                    :", total_sentencas_documento)

Total de documentos processados                                                : 20
Total de documentos com sentenças com uma palavra                              : 0
Total de documentos com pelo menos uma sentença com uma palavra(sem sentenciar): 0
Total de documentos excluídos                                                  : 0
Total sentenças nos documentos usando spaCy                                    : 20


### 4.5.3 Especifica os nomes dos arquivos de dados originais



In [52]:
# Nome do arquivo
NOME_ARQUIVO_ORIGINAL = "original.csv"
NOME_ARQUIVO_ORIGINAL_COMPACTADO = "original.zip"

### 4.5.4 Cria o arquivo dos documentos originais

In [53]:
# Import das bibliotecas.
import pandas as pd

# Cria o dataframe da lista
df_lista_documentos_originais = pd.DataFrame(lista_documentos_originais, columns = ["id","sentencas","documento"])
 
df_lista_documentos_originais.to_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_ORIGINAL, sep=";", index=False)

In [54]:
print(len(df_lista_documentos_originais))

20


In [55]:
df_lista_documentos_originais.sample(5)

,id,sentencas,documento
10,11,[What is a queue and how to dequeue an element...,What is a queue and how to dequeue an element ...
7,8,[How to pop elements in a stack data structure?],How to pop elements in a stack data structure?
15,16,[What is a queue and how to enqueue and dequeu...,What is a queue and how to enqueue and dequeue...
13,14,[What is a stack and how to push an element on...,What is a stack and how to push an element on it?
6,7,[How to pop elements from a stack?],How to pop elements from a stack?


### 4.5.5 Compacta e copia o arquivo original para uma pasta do GoogleDrive

Compacta os arquivos.

Usa o zip para compactar:
*   `-o` sobrescreve o arquivo se existir
*   `-j` Não cria nenhum diretório
*   `-q` Desliga as mensagens 


In [56]:
!zip -o -q -j "$DIRETORIO_LOCAL$NOME_ARQUIVO_ORIGINAL_COMPACTADO" "$DIRETORIO_LOCAL$NOME_ARQUIVO_ORIGINAL"

Copia o arquivo compactado para o GoogleDrive



In [57]:
# Se estiver executando no Google Colaboratory
if IN_COLAB:
    # Copia o arquivo original   
    !cp "$DIRETORIO_LOCAL$NOME_ARQUIVO_ORIGINAL_COMPACTADO" "$DIRETORIO_DRIVE"

    logging.info("Terminei a cópia")

INFO:root:Terminei a cópia


### 4.5.6 Carrega os dados

Carrega os dados das documentos a partir do arquivo.

In [58]:
# Import das bibliotecas.
import pandas as pd

# Abre o arquivo e retorna o DataFrame
df_lista_documentos_originais = pd.read_csv(DIRETORIO_LOCAL + NOME_ARQUIVO_ORIGINAL, sep=";", encoding="UTF-8")

In [59]:
df_lista_documentos_originais.sample(5)

,id,sentencas,documento
8,9,['What is a stack and how to push its element?'],What is a stack and how to push its element?
16,17,['How are the operations to push and pop eleme...,How are the operations to push and pop element...
15,16,['What is a queue and how to enqueue and deque...,What is a queue and how to enqueue and dequeue...
7,8,['How to pop elements in a stack data structur...,How to pop elements in a stack data structure?
0,1,['How to enqueue elements in a queue?'],How to enqueue elements in a queue?


# 5 Finalização

## 5.1 Tempo final de processamento



In [60]:
# Pega o tempo atual menos o tempo do início do processamento.
final_processamento = time.time()
tempo_total_processamento = formataTempo(final_processamento - inicio_processamento)

print("")
print("  Tempo processamento:  {:} (h:mm:ss)".format(tempo_total_processamento))


  Tempo processamento:  0:02:34 (h:mm:ss)
